In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.feature_selection import f_regression

%pip install tsfresh==0.20.1
%pip install scipy==1.12.0

# Load y from Cui et al.'s formation data

In [ ]:
filepath = f"""one_time_features_110323.csv"""
csvFile = pd.read_csv(filepath)
filepath = f"""protocol_params.xlsx"""
df_protocol_params = pd.read_excel(filepath)

In [ ]:
y = csvFile.regu_life

In [ ]:
ind_y = np.array([x for x in range(np.size(csvFile,0))])
ind_y = ind_y[~np.isnan(y)]
seq_num = csvFile.seq_num[ind_y]
y = y[ind_y]

ind_y = ind_y[seq_num != 206] # error loading Nova_Formation-206.csv
seq_num = np.array(seq_num[ind_y]) # error loading Nova_Formation-206.csv
y = np.array(y[ind_y])
log_y = np.log(y)

In [ ]:
len(ind_y)

In [ ]:
import pandas as pd
import os

df_A_Q_V_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/A_Q_V_all.xlsx")
df_A_t_V_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/A_t_V_all.xlsx")
df_B_Q_V_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/B_Q_V_all.xlsx")
df_B_V_t_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/B_V_t_all.xlsx")
df_C_Q_V_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/C_Q_V_all.xlsx")
df_C_V_t_all = pd.read_excel(os.path.dirname(os.getcwd()) + "/data_formation/C_V_t_all.xlsx")


In [ ]:
A_Q_V_all = df_A_Q_V_all.iloc[:, 1:-3].to_numpy()
A_t_V_all = df_A_t_V_all.iloc[:, 1:-3].to_numpy()
B_Q_V_all = df_B_Q_V_all.iloc[:, 1:-3].to_numpy()
B_V_t_all = df_B_V_t_all.iloc[:, 1:-3].to_numpy()
C_Q_V_all = df_C_Q_V_all.iloc[:, 1:-3].to_numpy()
C_V_t_all = df_C_V_t_all.iloc[:, 1:-3].to_numpy()

# Should group formation protocols by protocol parameters

In [ ]:
# CC1, CC2, CV, n are enough for group_labels.
group_label = np.array(list(df_protocol_params.groupby(by = ["CC1", "CC2", "CV", "n", "T", "t_OCV"]).ngroup()))
max(group_label)
pd.Series(data=df_protocol_params.groupby(by = ["CC1", "CC2", "CV", "n", "T", "t_OCV"]).ngroup()).to_excel("group_label.xlsx")

In [ ]:
id = np.repeat([x+1 for x in range(len(ind_y))], 1000)#.astype(int)
V_ind = np.tile(1.5 + (4.4-1.5)/999 * np.array([x for x in range(1000)]).astype(int), len(ind_y))

A_Q_V_tsfresh = np.c_[id, V_ind, np.reshape(A_Q_V_all, (-1, 1))]
A_t_V_tsfresh = np.c_[id, V_ind, np.reshape(A_t_V_all, (-1, 1))]

A_Q_V_tsfresh_df = pd.DataFrame(A_Q_V_tsfresh, columns=['id', 'V', 'Q_V'])
A_t_V_tsfresh_df = pd.DataFrame(A_t_V_tsfresh, columns=['id', 'V', 't_V'])

id = np.repeat([x+1 for x in range(len(ind_y))], 1000)#.astype(int)
V_ind = np.tile(4.4 - (4.4-3.0)/999 * np.array([x for x in range(1000)]).astype(int), len(ind_y))
t_ind = np.tile(np.array([x for x in range(1000)]).astype(int), len(ind_y))
B_Q_V_tsfresh = np.c_[id, V_ind, np.reshape(B_Q_V_all, (-1, 1))]
B_V_t_tsfresh = np.c_[id, t_ind, np.reshape(B_V_t_all, (-1, 1))]

B_Q_V_tsfresh_df = pd.DataFrame(B_Q_V_tsfresh, columns=['id', 'V', 'Q_V'])
B_V_t_tsfresh_df = pd.DataFrame(B_V_t_tsfresh, columns=['id', 't', 'V_t'])

id = np.repeat([x+1 for x in range(len(ind_y))], 1000)#.astype(int)
V_ind = np.tile(4.4 - (4.4-3.0)/999 * np.array([x for x in range(1000)]).astype(int), len(ind_y))
t_ind = np.tile(np.array([x for x in range(1000)]).astype(int), len(ind_y))
C_Q_V_tsfresh = np.c_[id, V_ind, np.reshape(C_Q_V_all, (-1, 1))]
C_V_t_tsfresh = np.c_[id, t_ind, np.reshape(C_V_t_all, (-1, 1))]

C_Q_V_tsfresh_df = pd.DataFrame(C_Q_V_tsfresh, columns=['id', 'V', 'Q_V'])
C_V_t_tsfresh_df = pd.DataFrame(C_V_t_tsfresh, columns=['id', 't', 'V_t'])


# Generate folders if not exist

In [ ]:
import os
if not os.path.exists(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML"):
    os.mkdir(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML")
if not os.path.exists(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled"):
    os.mkdir(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled")
if not os.path.exists(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist"):
    os.mkdir(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist")

# Extract features from each f(V) and f(t) in region A (first charge: CC1-CC2) using tsfresh

In [ ]:
from tsfresh import extract_features
A_Q_V_extracted_features = extract_features(A_Q_V_tsfresh_df, column_id="id", column_sort="V")
A_t_V_extracted_features = extract_features(A_t_V_tsfresh_df, column_id="id", column_sort="V")
B_Q_V_extracted_features = extract_features(B_Q_V_tsfresh_df, column_id="id", column_sort="V")
B_V_t_extracted_features = extract_features(B_V_t_tsfresh_df, column_id="id", column_sort="t")
C_Q_V_extracted_features = extract_features(C_Q_V_tsfresh_df, column_id="id", column_sort="V")
C_V_t_extracted_features = extract_features(C_V_t_tsfresh_df, column_id="id", column_sort="t")

In [ ]:
A_Q_V_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/A_Q_V_extracted_features.xlsx")
A_t_V_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/A_t_V_extracted_features.xlsx")
B_Q_V_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/B_Q_V_extracted_features.xlsx")
B_V_t_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/B_V_t_extracted_features.xlsx")
C_Q_V_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/C_Q_V_extracted_features.xlsx")
C_V_t_extracted_features.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_fulllist/C_V_t_extracted_features.xlsx")

# Pre-screen features that are constant

In [ ]:
from tsfresh.utilities.dataframe_functions import impute
impute(A_Q_V_extracted_features)
impute(A_t_V_extracted_features)
impute(B_Q_V_extracted_features)
impute(B_V_t_extracted_features)
impute(C_Q_V_extracted_features)
impute(C_V_t_extracted_features)

A_Q_V_index_notconstant = np.where(A_Q_V_extracted_features.max(axis=0) != A_Q_V_extracted_features.min(axis=0))[0]
_, A_Q_V_p_values = f_regression(A_Q_V_extracted_features[A_Q_V_extracted_features.columns[A_Q_V_index_notconstant]], y.reshape(-1, 1))
_, log_A_Q_V_p_values = f_regression(A_Q_V_extracted_features[A_Q_V_extracted_features.columns[A_Q_V_index_notconstant]], log_y.reshape(-1, 1))
A_t_V_index_notconstant = np.where(A_t_V_extracted_features.max(axis=0) != A_t_V_extracted_features.min(axis=0))[0]
_, A_t_V_p_values = f_regression(A_t_V_extracted_features[A_t_V_extracted_features.columns[A_t_V_index_notconstant]], y.reshape(-1, 1))
_, log_A_t_V_p_values = f_regression(A_t_V_extracted_features[A_t_V_extracted_features.columns[A_t_V_index_notconstant]], log_y.reshape(-1, 1))
B_Q_V_index_notconstant = np.where(B_Q_V_extracted_features.max(axis=0) != B_Q_V_extracted_features.min(axis=0))[0]
_, B_Q_V_p_values = f_regression(B_Q_V_extracted_features[B_Q_V_extracted_features.columns[B_Q_V_index_notconstant]], y.reshape(-1, 1))
_, log_B_Q_V_p_values = f_regression(B_Q_V_extracted_features[B_Q_V_extracted_features.columns[B_Q_V_index_notconstant]], log_y.reshape(-1, 1))
B_V_t_index_notconstant = np.where(B_V_t_extracted_features.max(axis=0) != B_V_t_extracted_features.min(axis=0))[0]
_, B_V_t_p_values = f_regression(B_V_t_extracted_features[B_V_t_extracted_features.columns[B_V_t_index_notconstant]], y.reshape(-1, 1))
_, log_B_V_t_p_values = f_regression(B_V_t_extracted_features[B_V_t_extracted_features.columns[B_V_t_index_notconstant]], log_y.reshape(-1, 1))
C_Q_V_index_notconstant = np.where(C_Q_V_extracted_features.max(axis=0) != C_Q_V_extracted_features.min(axis=0))[0]
_, C_Q_V_p_values = f_regression(C_Q_V_extracted_features[C_Q_V_extracted_features.columns[C_Q_V_index_notconstant]], y.reshape(-1, 1))
_, log_C_Q_V_p_values = f_regression(C_Q_V_extracted_features[C_Q_V_extracted_features.columns[C_Q_V_index_notconstant]], log_y.reshape(-1, 1))
C_V_t_index_notconstant = np.where(C_V_t_extracted_features.max(axis=0) != C_V_t_extracted_features.min(axis=0))[0]
_, C_V_t_p_values = f_regression(C_V_t_extracted_features[C_V_t_extracted_features.columns[C_V_t_index_notconstant]], y.reshape(-1, 1))
_, log_C_V_t_p_values = f_regression(C_V_t_extracted_features[C_V_t_extracted_features.columns[C_V_t_index_notconstant]], log_y.reshape(-1, 1))


In [ ]:
fig, axs = plt.subplots(figsize=(30,10), nrows=2, ncols=6)

logbins = np.logspace(-8,0,17)

axs[0,0].hist(A_Q_V_p_values, bins=logbins)
axs[0,0].set_xscale('log')
axs[0,0].minorticks_on()
axs[0,0].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,0].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,0].set_xlabel('p-values')
axs[0,0].set_ylabel('#')
axs[0,0].set_title('p values in Q^{A}(V)')

axs[0,1].hist(A_t_V_p_values, bins=logbins)
axs[0,1].set_xscale('log')
axs[0,1].minorticks_on()
axs[0,1].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,1].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,1].set_xlabel('p-values')
axs[0,1].set_ylabel('#')
axs[0,1].set_title('p values in t^{A}(V)')

axs[0,2].hist(B_Q_V_p_values, bins=logbins)
axs[0,2].set_xscale('log')
axs[0,2].minorticks_on()
axs[0,2].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,2].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,2].set_xlabel('p-values')
axs[0,2].set_ylabel('#')
axs[0,2].set_title('p values in Q^{B}(V)')

axs[0,3].hist(B_V_t_p_values, bins=logbins)
axs[0,3].set_xscale('log')
axs[0,3].minorticks_on()
axs[0,3].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,3].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,3].set_xlabel('p-values')
axs[0,3].set_ylabel('#')
axs[0,3].set_title('p values in V^{B}(t)')

axs[0,4].hist(C_Q_V_p_values, bins=logbins)
axs[0,4].set_xscale('log')
axs[0,4].minorticks_on()
axs[0,4].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,4].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,4].set_xlabel('p-values')
axs[0,4].set_ylabel('#')
axs[0,4].set_title('p values in Q^{C}(V)')

axs[0,5].hist(C_V_t_p_values, bins=logbins)
axs[0,5].set_xscale('log')
axs[0,5].minorticks_on()
axs[0,5].grid(which='major', linestyle='-', linewidth='0.5')
axs[0,5].grid(which='minor', linestyle=':', linewidth='0.5')
axs[0,5].set_xlabel('p-values')
axs[0,5].set_ylabel('#')
axs[0,5].set_title('p values in V^{C}(t)')

axs[1,0].hist(log_A_Q_V_p_values, bins=logbins)
axs[1,0].set_xscale('log')
axs[1,0].minorticks_on()
axs[1,0].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,0].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,0].set_xlabel('p-values')
axs[1,0].set_ylabel('#')
axs[1,0].set_title('p values in Q^{A}(V) with log_y')

axs[1,1].hist(log_A_t_V_p_values, bins=logbins)
axs[1,1].set_xscale('log')
axs[1,1].minorticks_on()
axs[1,1].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,1].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,1].set_xlabel('p-values')
axs[1,1].set_ylabel('#')
axs[1,1].set_title('p values in t^{A}(V) with log_y')

axs[1,2].hist(log_B_Q_V_p_values, bins=logbins)
axs[1,2].set_xscale('log')
axs[1,2].minorticks_on()
axs[1,2].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,2].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,2].set_xlabel('p-values')
axs[1,2].set_ylabel('#')
axs[1,2].set_title('p values in Q^{B}(V) with log_y')

axs[1,3].hist(log_B_V_t_p_values, bins=logbins)
axs[1,3].set_xscale('log')
axs[1,3].minorticks_on()
axs[1,3].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,3].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,3].set_xlabel('p-values')
axs[1,3].set_ylabel('#')
axs[1,3].set_title('p values in V^{B}(t) with log_y')

axs[1,4].hist(log_C_Q_V_p_values, bins=logbins)
axs[1,4].set_xscale('log')
axs[1,4].minorticks_on()
axs[1,4].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,4].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,4].set_xlabel('p-values')
axs[1,4].set_ylabel('#')
axs[1,4].set_title('p values in Q^{C}(V) with log_y')

axs[1,5].hist(log_C_V_t_p_values, bins=logbins)
axs[1,5].set_xscale('log')
axs[1,5].minorticks_on()
axs[1,5].grid(which='major', linestyle='-', linewidth='0.5')
axs[1,5].grid(which='minor', linestyle=':', linewidth='0.5')
axs[1,5].set_xlabel('p-values')
axs[1,5].set_ylabel('#')
axs[1,5].set_title('p values in V^{C}(t) with log_y')

# Filter features by using univariate test followed by tsfresh.select_features

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

alpha_list_log = np.linspace(-8,0,17)

for alpha in alpha_list_log:
    
    A_Q_V_features_univariate = A_Q_V_extracted_features[A_Q_V_extracted_features.columns[A_Q_V_index_notconstant[A_Q_V_p_values < 10 ** alpha]]]
    A_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/A_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    A_Q_V_features_univariate_SPA = A_Q_V_features_univariate
    A_Q_V_features_univariate_SPA['y'] = y
    A_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/A_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    A_Q_V_features_univariate_tsfresh = select_features(A_Q_V_features_univariate, y)
    A_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/A_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    A_Q_V_features_univariate_tsfresh_SPA = A_Q_V_features_univariate_tsfresh
    A_Q_V_features_univariate_tsfresh_SPA['y'] = y
    A_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/A_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    A_t_V_features_univariate = A_t_V_extracted_features[A_t_V_extracted_features.columns[A_t_V_index_notconstant[A_t_V_p_values < 10 ** alpha]]]
    A_t_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/A_t_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    A_t_V_features_univariate_SPA = A_t_V_features_univariate
    A_t_V_features_univariate_SPA['y'] = y
    A_t_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/A_t_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    A_t_V_features_univariate_tsfresh = select_features(A_t_V_features_univariate, y)
    A_t_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/A_t_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    A_t_V_features_univariate_tsfresh_SPA = A_t_V_features_univariate_tsfresh
    A_t_V_features_univariate_tsfresh_SPA['y'] = y
    A_t_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/A_t_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)

    B_Q_V_features_univariate = B_Q_V_extracted_features[B_Q_V_extracted_features.columns[B_Q_V_index_notconstant[B_Q_V_p_values < 10 ** alpha]]]
    B_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/B_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    B_Q_V_features_univariate_SPA = B_Q_V_features_univariate
    B_Q_V_features_univariate_SPA['y'] = y
    B_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/B_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    B_Q_V_features_univariate_tsfresh = select_features(B_Q_V_features_univariate, y)
    B_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/B_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    B_Q_V_features_univariate_tsfresh_SPA = B_Q_V_features_univariate_tsfresh
    B_Q_V_features_univariate_tsfresh_SPA['y'] = y
    B_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/B_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    B_V_t_features_univariate = B_V_t_extracted_features[B_V_t_extracted_features.columns[B_V_t_index_notconstant[B_V_t_p_values < 10 ** alpha]]]
    B_V_t_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/B_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx")
    B_V_t_features_univariate_SPA = B_V_t_features_univariate
    B_V_t_features_univariate_SPA['y'] = y
    B_V_t_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/B_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    B_V_t_features_univariate_tsfresh = select_features(B_V_t_features_univariate, y)
    B_V_t_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/B_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    B_V_t_features_univariate_tsfresh_SPA = B_V_t_features_univariate_tsfresh
    B_V_t_features_univariate_tsfresh_SPA['y'] = y
    B_V_t_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/B_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)

    C_Q_V_features_univariate = C_Q_V_extracted_features[C_Q_V_extracted_features.columns[C_Q_V_index_notconstant[C_Q_V_p_values < 10 ** alpha]]]
    C_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/C_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    C_Q_V_features_univariate_SPA = C_Q_V_features_univariate
    C_Q_V_features_univariate_SPA['y'] = y
    C_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/C_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    C_Q_V_features_univariate_tsfresh = select_features(C_Q_V_features_univariate, y)
    C_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/C_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    C_Q_V_features_univariate_tsfresh_SPA = C_Q_V_features_univariate_tsfresh
    C_Q_V_features_univariate_tsfresh_SPA['y'] = y
    C_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/C_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    C_V_t_features_univariate = C_V_t_extracted_features[C_V_t_extracted_features.columns[C_V_t_index_notconstant[C_V_t_p_values < 10 ** alpha]]]
    C_V_t_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/C_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx")
    C_V_t_features_univariate_SPA = C_V_t_features_univariate
    C_V_t_features_univariate_SPA['y'] = y
    C_V_t_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/C_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    C_V_t_features_univariate_tsfresh = select_features(C_V_t_features_univariate, y)
    C_V_t_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/C_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    C_V_t_features_univariate_tsfresh_SPA = C_V_t_features_univariate_tsfresh
    C_V_t_features_univariate_tsfresh_SPA['y'] = y
    C_V_t_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/C_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)

    log_A_Q_V_features_univariate = A_Q_V_extracted_features[A_Q_V_extracted_features.columns[A_Q_V_index_notconstant[log_A_Q_V_p_values < 10 ** alpha]]]
    log_A_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_A_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_A_Q_V_features_univariate_SPA = log_A_Q_V_features_univariate
    log_A_Q_V_features_univariate_SPA['y'] = log_y
    log_A_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_A_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_A_Q_V_features_univariate_tsfresh = select_features(log_A_Q_V_features_univariate, log_y)
    log_A_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_A_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_A_Q_V_features_univariate_tsfresh_SPA = log_A_Q_V_features_univariate_tsfresh
    log_A_Q_V_features_univariate_tsfresh_SPA['y'] = log_y
    log_A_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_A_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    log_A_t_V_features_univariate = A_t_V_extracted_features[A_t_V_extracted_features.columns[A_t_V_index_notconstant[log_A_t_V_p_values < 10 ** alpha]]]
    log_A_t_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_A_t_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_A_t_V_features_univariate_SPA = log_A_t_V_features_univariate
    log_A_t_V_features_univariate_SPA['y'] = log_y
    log_A_t_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_A_t_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_A_t_V_features_univariate_tsfresh = select_features(log_A_t_V_features_univariate, log_y)
    log_A_t_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_A_t_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_A_t_V_features_univariate_tsfresh_SPA = log_A_t_V_features_univariate_tsfresh
    log_A_t_V_features_univariate_tsfresh_SPA['y'] = log_y
    log_A_t_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_A_t_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)

    log_B_Q_V_features_univariate = B_Q_V_extracted_features[B_Q_V_extracted_features.columns[B_Q_V_index_notconstant[log_B_Q_V_p_values < 10 ** alpha]]]
    log_B_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_B_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_B_Q_V_features_univariate_SPA = log_B_Q_V_features_univariate
    log_B_Q_V_features_univariate_SPA['y'] = log_y
    log_B_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_B_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_B_Q_V_features_univariate_tsfresh = select_features(log_B_Q_V_features_univariate, log_y)
    log_B_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_B_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_B_Q_V_features_univariate_tsfresh_SPA = log_B_Q_V_features_univariate_tsfresh
    log_B_Q_V_features_univariate_tsfresh_SPA['y'] = log_y
    log_B_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_B_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    log_B_V_t_features_univariate = B_V_t_extracted_features[B_V_t_extracted_features.columns[B_V_t_index_notconstant[log_B_V_t_p_values < 10 ** alpha]]]
    log_B_V_t_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_B_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_B_V_t_features_univariate_SPA = log_B_V_t_features_univariate
    log_B_V_t_features_univariate_SPA['y'] = log_y
    log_B_V_t_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_B_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_B_V_t_features_univariate_tsfresh = select_features(log_B_V_t_features_univariate, log_y)
    log_B_V_t_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_B_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_B_V_t_features_univariate_tsfresh_SPA = log_B_V_t_features_univariate_tsfresh
    log_B_V_t_features_univariate_tsfresh_SPA['y'] = log_y
    log_B_V_t_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_B_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)

    log_C_Q_V_features_univariate = C_Q_V_extracted_features[C_Q_V_extracted_features.columns[C_Q_V_index_notconstant[log_C_Q_V_p_values < 10 ** alpha]]]
    log_C_Q_V_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_C_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_C_Q_V_features_univariate_SPA = log_C_Q_V_features_univariate
    log_C_Q_V_features_univariate_SPA['y'] = log_y
    log_C_Q_V_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_C_Q_V_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_C_Q_V_features_univariate_tsfresh = select_features(log_C_Q_V_features_univariate, log_y)
    log_C_Q_V_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_C_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_C_Q_V_features_univariate_tsfresh_SPA = log_C_Q_V_features_univariate_tsfresh
    log_C_Q_V_features_univariate_tsfresh_SPA['y'] = log_y
    log_C_Q_V_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_C_Q_V_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    
    log_C_V_t_features_univariate = C_V_t_extracted_features[C_V_t_extracted_features.columns[C_V_t_index_notconstant[log_C_V_t_p_values < 10 ** alpha]]]
    log_C_V_t_features_univariate.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_C_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx")
    log_C_V_t_features_univariate_SPA = log_C_V_t_features_univariate
    log_C_V_t_features_univariate_SPA['y'] = log_y
    log_C_V_t_features_univariate_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_C_V_t_features_univariate_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    log_C_V_t_features_univariate_tsfresh = select_features(log_C_V_t_features_univariate, log_y)
    log_C_V_t_features_univariate_tsfresh.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML_labeled/log_C_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx")
    log_C_V_t_features_univariate_tsfresh_SPA = log_C_V_t_features_univariate_tsfresh
    log_C_V_t_features_univariate_tsfresh_SPA['y'] = log_y
    log_C_V_t_features_univariate_tsfresh_SPA.to_excel(os.path.dirname(os.getcwd()) + "/Features_tsfresh_autoML/log_C_V_t_features_univariate_tsfresh_" + str(abs(alpha)) + ".xlsx", header=False, index=False)
    